# Reducing ASD wavelengths error correlation matrix from `float64` to `float32`

Javier Gatón Herguedas

## Introduction

When interpolating the reflectance from the **CIMEL** points to the whole **ASD** spectrum, the ASD wavelengths error
correlation matrix is used to propagate the uncertainties. This is currently a **2151 × 2151** double-precision
floating-point (`float64`) matrix.

The `comet_maths` library requires **M** to be **positive semi-definite (PSD)**, which can be described
as **M** being a **Hermitian matrix** (including real symmetric matrices) where all its **eigenvalues are
real and non-negative**.

If the provided matrix is not **positive semi-definite**, `comet_maths` calculates the **closest positive
definite matrix** during run-time, which is a **time-consuming task** for a **2151 × 2151** matrix.

Using a **`float32` ASD error correlation matrix** instead of `float64` speeds up the simulation **2.5 times**. However, 
simply converting values to `float32` leads to truncation, causing the matrix to **lose its positive semi-definite property**. 
If we then compute the **nearest positive definite matrix** using `comet_maths.nearestPD_cholesky`, the resulting matrix has 
**diagonal values greater than 1**, which is **incompatible** with other uncertainty propagation functions.

### Objective
In this notebook, we explore different methods to obtain a **positive semi-definite `float32` matrix** 
with **1s in the diagonal**, derived from the original **`float64` positive semi-definite matrix**.


## Imports, data loads, and base function definitions

In [1]:
import warnings

import numpy as np
import xarray as xr
import comet_maths as cm

warnings.filterwarnings("ignore", message="Duplicate dimension names present")
warnings.filterwarnings("ignore", message="One of the provided covariance")

In [2]:
_DS_ASD_PATH = "./ds_ASD_original.nc"


def get_err_corr() -> np.ndarray:
    with xr.open_dataset(_DS_ASD_PATH) as ds:
        vals = ds["err_corr_reflectance_wavelength"].values
    return vals

In [3]:
def is_PSD(mat: np.ndarray, tol: float = 1e-10):
    """Check if a matrix is Positive Semi-Definite (PSD) by verifying eigenvalues."""
    eigenvalues = np.linalg.eigvalsh(mat)
    lim = -tol  # Allow for small numerical errors
    return np.all(eigenvalues >= lim)

In [4]:
# We need some tolerance due to floating point arithmetic.
# Even the original matrix wouldn't pass the is_PSD test without it
print(is_PSD(get_err_corr(), tol=0))
print(is_PSD(get_err_corr()))

False
True


In [5]:
def normalise_div(A: np.ndarray):
    return A / np.max(A)


def normalise(A: np.ndarray):
    A = A.copy()
    D = np.diag(1 / np.sqrt(np.diag(A)))
    A = D @ A @ D
    np.fill_diagonal(A, 1)
    return A

In [6]:
def analyze_ec(ec: np.ndarray, tol: float = 1e-10) -> dict:
    ec_src = get_err_corr()
    rel_diff = 100 * (ec - ec_src) / ec_src
    rel_diff = np.abs(np.ravel(rel_diff))
    analysis = {
        "dtype": ec.dtype,
        "PSD": is_PSD(ec, tol),
        "diff_min": min(rel_diff),
        "diff_max": max(rel_diff),
        "diff_mean": np.mean(rel_diff),
        "diff_std": np.std(rel_diff),
        "max_diag": max(np.diag(ec)),
    }
    return analysis


def describe_ec(ec: np.ndarray, tol: float = 1e-10):
    analysis = analyze_ec(ec, tol)
    vals = []
    for k in analysis:
        if k.startswith("diff"):
            vals.append(f"{k}: {analysis[k]:.4g}%")
        else:
            vals.append(f"{k}: {analysis[k]}")
    vals = "\n".join(vals)
    print(vals)

## Using comet_maths.nearestPD_cholesky

As described in the introduction, the original matrix is PSD

In [7]:
ec = get_err_corr()
describe_ec(ec)

dtype: float64
PSD: True
diff_min: 0%
diff_max: 0%
diff_mean: 0%
diff_std: 0%
max_diag: 1.0000000000000002


But the `float32`-truncated matrix is not PSD

In [8]:
ec = get_err_corr().astype(np.float32)
describe_ec(ec)

dtype: float32
PSD: False
diff_min: 0%
diff_max: 3.424e-06%
diff_mean: 1.528e-06%
diff_std: 8.844e-07%
max_diag: 1.0


Using `comet_maths.nearestPD_cholesky` we get a PSD matrix, but the diagonal contains values too big (slightly over 1)

In [9]:
ec = get_err_corr().astype(np.float32)
ec = cm.nearestPD_cholesky(ec, return_cholesky=False)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 2.783e-13%
diff_max: 0.01296%
diff_mean: 1.325e-05%
diff_std: 0.0002786%
max_diag: 1.0001295804977417


If we normalise that matrix, we get a valid PSD matrix.

In [10]:
ec = get_err_corr().astype(np.float32)
ec = cm.nearestPD_cholesky(ec, return_cholesky=False)
ec = normalise(ec)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.01298%
diff_mean: 0.01291%
diff_std: 0.0002787%
max_diag: 1.0


The mean relative difference is 0.01291%. Can we get a more similar PSD matrix?

## Using a nearestPSD method

The most popular method is a similar version of the comet_maths method, but there are multiple
methods online for the calculation of the nearest PSD. Most of them haven't been useful for the
purpose of this notebook, except for a modification of the method described in the Method 0 subsection.

### Method 0

In [11]:
def closestPSD_0(A: np.ndarray, tol=1e-10) -> np.ndarray:
    A = (A + A.T) / 2
    eigvals, eigvecs = np.linalg.eigh(A)
    eigvals[eigvals < tol] = 0
    A = eigvecs @ np.diag(eigvals) @ eigvecs.T
    return A

In [12]:
ec = get_err_corr().astype(np.float32)
ec = closestPSD_0(ec)
describe_ec(ec)

dtype: float32
PSD: False
diff_min: 7.873e-13%
diff_max: 4.768e-05%
diff_mean: 4.343e-06%
diff_std: 3.26e-06%
max_diag: 1.0000004768371582


It doesn't work.

If we look at Brian Borchers' answer in stackexchange: https://scicomp.stackexchange.com/a/12984,
we see that a good definition for the `is_PSD` method is to, instead of checking that
all eigenvalues are greater than `-tol`, check that they are greater than `-tol * max(eigenvals)`.

We aren't modifying our `is_PSD` method, but if we apply that idea to this function it starts working.
This can be seen in the Method 1 subsection.

### Method 1

In [13]:
def closestPSD_maxeigval(A: np.ndarray, tol=1e-10) -> np.ndarray:
    A = (A + A.T) / 2
    eigvals, eigvecs = np.linalg.eigh(A)
    eigvals[eigvals < tol] = tol * max(eigvals)
    A = eigvecs @ np.diag(eigvals) @ eigvecs.T
    return A

We increase the tolerance until the resulting matrix is actually PSD.

The diagonal values are too high.

In [14]:
ec = get_err_corr().astype(np.float32)
for i in range(10):
    eci = closestPSD_maxeigval(ec, tol=10 ** (-10 + i))
    if is_PSD(eci):
        break
ec = eci
print(f"1e-{10-i}")
describe_ec(ec)

1e-6
dtype: float32
PSD: True
diff_min: 5.595e-10%
diff_max: 0.1881%
diff_mean: 0.001369%
diff_std: 0.003923%
max_diag: 1.0018811225891113


Normalising the matrix we get a valid PSD matrix but the mean relative difference is much worse than with the closestPD
method.

In [15]:
ec = get_err_corr().astype(np.float32)
ec = closestPSD_maxeigval(ec, tol=1e-6)
ec = normalise(ec)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.1916%
diff_mean: 0.1755%
diff_std: 0.01072%
max_diag: 1.0


If the function is applied to the original matrix before applying it to the truncated one, we
obtain slightly better results.

In [16]:
ec = get_err_corr()
ec = closestPSD_maxeigval(ec)
describe_ec(ec)
ec = ec.astype(np.float32)
describe_ec(ec)
for i in range(10):
    eci = closestPSD_maxeigval(ec, tol=10 ** (-10 + i))
    if is_PSD(eci):
        break
ec = eci
print(f"1e-{10-i}")
describe_ec(ec)
ec = normalise(ec)
describe_ec(ec)

dtype: float64
PSD: True
diff_min: 0%
diff_max: 2.091e-05%
diff_mean: 4.777e-08%
diff_std: 4.392e-07%
max_diag: 1.000000209136463
dtype: float32
PSD: False
diff_min: 2.783e-13%
diff_max: 2.384e-05%
diff_mean: 1.54e-06%
diff_std: 9.982e-07%
max_diag: 1.000000238418579
1e-6
dtype: float32
PSD: True
diff_min: 7.622e-10%
diff_max: 0.1707%
diff_mean: 0.001616%
diff_std: 0.003625%
max_diag: 1.0017071962356567
dtype: float32
PSD: False
diff_min: 0%
diff_max: 0.1779%
diff_mean: 0.1595%
diff_std: 0.009314%
max_diag: 1.0


If we apply the function to the original matrix but with an increased tolerance, the results improve.

This was discovered setting the tolerance to 1e-8, and manually through trial and error the value 
4.772e-9 was selected.
A valid PSD matrix with a mean relative difference of 0.0009364% was obtained. This is over 10 times
smaller than the matrix adapted initally.

In [17]:
ec = get_err_corr()
ec = closestPSD_maxeigval(ec, 4.772e-9)  # 4.772e-9 0.0009364%
describe_ec(ec)
ec = ec.astype(np.float32)
describe_ec(ec)
if not is_PSD(ec):
    for i in range(10):
        for j in range(9):
            eci = closestPSD_maxeigval(ec, tol=(j + 1) * 10 ** (-10 + i))
            if is_PSD(eci):
                break
        if is_PSD(eci):
            break
    ec = eci
    print(f"{(j+1)}e-{10-i}")
    describe_ec(ec)
ec = normalise(ec)
describe_ec(ec)
if not is_PSD(ec):
    for i in range(10):
        for j in range(9):
            eci = closestPSD_maxeigval(ec, tol=(j + 1) * 10 ** (-10 + i))
            eci = normalise(eci)
            if is_PSD(eci):
                break
        if is_PSD(eci):
            break
    ec = eci
    print(f"{j+1}e-{10-i}")
    describe_ec(ec)
describe_ec(ec)

dtype: float64
PSD: True
diff_min: 7.269e-14%
diff_max: 0.000998%
diff_mean: 2.28e-06%
diff_std: 2.096e-05%
max_diag: 1.0000099799920181
dtype: float32
PSD: True
diff_min: 2.783e-13%
diff_max: 0.001001%
diff_mean: 3.087e-06%
diff_std: 2.092e-05%
max_diag: 1.0000100135803223
dtype: float32
PSD: False
diff_min: 0%
diff_max: 0.001011%
diff_mean: 0.0009467%
diff_std: 5.769e-05%
max_diag: 1.0
2e-10
dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.001021%
diff_mean: 0.0009364%
diff_std: 5.816e-05%
max_diag: 1.0
dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.001021%
diff_mean: 0.0009364%
diff_std: 5.816e-05%
max_diag: 1.0


## Store the new matrix

In [18]:
ec = get_err_corr()
ec = closestPSD_maxeigval(ec, tol=4.772e-9)
ec = ec.astype(np.float32)
ec = normalise(ec)
ec = closestPSD_maxeigval(ec, tol=2e-10)
ec = normalise(ec)
describe_ec(ec)

dtype: float32
PSD: True
diff_min: 0%
diff_max: 0.001021%
diff_mean: 0.0009364%
diff_std: 5.816e-05%
max_diag: 1.0


In [19]:
ds = xr.open_dataset(_DS_ASD_PATH)
ds["err_corr_reflectance_wavelength"].values = ec
ds.to_netcdf("./ds_ASD_32.nc")
ds.close()